In [1]:
pwd


u'/Users/vincent/OneDrive/axa/DS winter school/winter_school_DS_QH2017/prg'

read the data and import the package

In [18]:
import random
import numpy as np
import igraph
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.cross_validation import KFold
from sklearn.metrics import f1_score
from sklearn import preprocessing
import nltk
import csv

from scipy.spatial.distance import cosine
import networkx as nx



# nltk.download('punkt') # for tokenization
# nltk.download('stopwords')

stpwds = set(nltk.corpus.stopwords.words("english"))
stemmer = nltk.stem.PorterStemmer()

with open("../input/testing_set.txt", "r") as f:
    reader = csv.reader(f)
    testing_set  = list(reader)

testing_set = [element[0].split(" ") for element in testing_set]
# data loading and preprocessing

# the columns of the data frame below are:
# (1) paper unique ID (integer)
# (2) publication year (integer)
# (3) paper title (string)
# (4) authors (strings separated by ,)
# (5) name of journal (optional) (string)
# (6) abstract (string) - lowercased, free of punctuation except intra-word dashes

with open("../input/training_set.txt", "r") as f:
    reader = csv.reader(f)
    training_set  = list(reader)

training_set = [element[0].split(" ") for element in training_set]
with open("../input/node_information.csv", "r") as f:
    reader = csv.reader(f)
    node_info  = list(reader)

In [2]:
training_set[0:10]

[['9510123', '9502114', '1'],
 ['9707075', '9604178', '1'],
 ['9312155', '9506142', '0'],
 ['9911255', '302165', '0'],
 ['9701033', '209076', '0'],
 ['9710020', '9709228', '1'],
 ['9901042', '9510135', '1'],
 ['209146', '9502077', '0'],
 ['9705079', '9702201', '1'],
 ['3016', '9207067', '0']]

In [3]:
testing_set[0:10]

[['9807076', '9807139'],
 ['109162', '1182'],
 ['9702187', '9510135'],
 ['111048', '110115'],
 ['9910176', '9410073'],
 ['9164', '9612141'],
 ['9411033', '9412038'],
 ['9411095', '9304102'],
 ['9603027', '301251'],
 ['9709080', '9702161']]

In [4]:
node_info[0:2]

[['1001',
  '2000',
  'compactification geometry and duality',
  'Paul S. Aspinwall',
  '',
  'these are notes based on lectures given at tasi99 we review the geometry of the moduli space of n 2 theories in four dimensions from the point of view of superstring compactification the cases of a type iia or type iib string compactified on a calabi-yau threefold and the heterotic string compactified on k3xt2 are each considered in detail we pay specific attention to the differences between n 2 theories and n 2 theories the moduli spaces of vector multiplets and the moduli spaces of hypermultiplets are reviewed in the case of hypermultiplets this review is limited by the poor state of our current understanding some peculiarities such as mixed instantons and the non-existence of a universal hypermultiplet are discussed'],
 ['1002',
  '2000',
  'domain walls and massive gauged supergravity potentials',
  'M. Cvetic, H. Lu, C.N. Pope',
  'Class.Quant.Grav.',
  'we point out that massive gauged 

In [19]:
#to test code we select sample

random.seed(22)

to_keep = random.sample(range(len(training_set)), k=int(round(len(training_set)*0.05)))
training_set = [training_set[i] for i in to_keep]
valid_ids=set()
for element in training_set:
    valid_ids.add(element[0])
    valid_ids.add(element[1])

tmp=[element for element in node_info if element[0] in valid_ids ]
node_info=tmp
del tmp



In [20]:
IDs = []
ID_pos={}
for element in node_info:
    ID_pos[element[0]]=len(IDs)
    IDs.append(element[0])


    
    


In [21]:
print ("build graph")
graph = nx.Graph()
edges = []
nodes = set()

build graph


In [22]:

for i in range(len(training_set)):
    source = training_set[i][0]
    target = training_set[i][1]
    nodes.add(source)
    nodes.add(target)
    if training_set[i][2]=="1":
        edges.append((source,target))


In [23]:
graph.add_nodes_from(nodes)
graph.add_edges_from(edges)

neighbor_count_source = []
neighbor_count_target = []


In [27]:
# we will use three basic features:

# number of overlapping words in title
overlap_title = []

# temporal distance between the papers
temp_diff = []

# number of common authors
comm_auth = []

#content text feauture
content_fea=[]

#tfidf similarities
tfidf_cos=[]

#similarity based on neighbors in graph
neighbor_sim=[]

In [36]:

vectorizer=TfidfVectorizer(stop_words="english"
            #,max_df=0.75
                          )

corpus=[element[5] for element in node_info]

M = vectorizer.fit_transform(corpus)




In [7]:
corpus

['these are notes based on lectures given at tasi99 we review the geometry of the moduli space of n 2 theories in four dimensions from the point of view of superstring compactification the cases of a type iia or type iib string compactified on a calabi-yau threefold and the heterotic string compactified on k3xt2 are each considered in detail we pay specific attention to the differences between n 2 theories and n 2 theories the moduli spaces of vector multiplets and the moduli spaces of hypermultiplets are reviewed in the case of hypermultiplets this review is limited by the poor state of our current understanding some peculiarities such as mixed instantons and the non-existence of a universal hypermultiplet are discussed',
 'we point out that massive gauged supergravity potentials for example those arising due to the massive breathing mode of sphere reductions in m-theory or string theory allow for supersymmetric static domain wall solutions which are a hybrid of a randall-sundrum doma

In [42]:
type(corpus)

list

In [43]:
type(vectorizer)

sklearn.feature_extraction.text.TfidfVectorizer

In [44]:
vectorizer

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [35]:
counter = 0
for i in xrange(len(training_set)):
    source = training_set[i][0]
    target = training_set[i][1]

    source_info = node_info[ID_pos[source]]
    target_info = node_info[ID_pos[target]]
  
    # convert to lowercase and tokenize
    source_title = source_info[2].lower().split(" ")
    # remove stopwords
    source_title = [token for token in source_title if token not in stpwds]
    source_title = [stemmer.stem(token) for token in source_title]
    
    target_title = target_info[2].lower().split(" ")
    target_title = [token for token in target_title if token not in stpwds]
    target_title = [stemmer.stem(token) for token in target_title]
    
    source_auth = source_info[3].split(",")
    target_auth = target_info[3].split(",")

    source_sim = M[ID_pos[source],:].toarray()[0]
    target_sim = M[ID_pos[target],:].toarray()[0]

    temp_cosine = 0.0
    if np.linalg.norm(source_sim)!=0 and np.linalg.norm(target_sim)!=0 :
        temp_cosine = cosine(source_sim,target_sim)
    tfidf_cos.append(temp_cosine)

    tfidf_cos.append(temp_cosine)
    neighbor_source = len(list(graph.neighbors(source)))
    neighbor_target = len(list(graph.neighbors(target)))
    neighbor_count_source.append(neighbor_source)
    neighbor_count_target.append(neighbor_target)
    neighbor_source = set(list(graph.neighbors(source)))
    neighbor_target = set(list(graph.neighbors(target)))
    neighbor_sim.append(len(neighbor_source.intersection(neighbor_target)))


 
    
    overlap_title.append(len(set(source_title).intersection(set(target_title))))
    temp_diff.append(int(source_info[1]) - int(target_info[1]))
    comm_auth.append(len(set(source_auth).intersection(set(target_auth))))


    if counter % 10000 == 0:
        print counter, "training examples processsed"
    counter += 1
# convert list of lists into array
# documents as rows, unique words as columns (i.e., example as rows, features as columns)
#training_features = np.array([overlap_title, temp_diff, comm_auth]).T

0 training examples processsed
10000 training examples processsed
20000 training examples processsed
30000 training examples processsed


In [34]:
# convert list of lists into array
# documents as rows, unique words as columns (i.e., example as rows, features as columns)
#training_features = np.array([overlap_title, temp_diff, comm_auth, tfidf_cos, neighbor_sim]).T
training_features = np.array([overlap_title, temp_diff, comm_auth, neighbor_sim,neighbor_count_source,neighbor_count_target,tfidf_cos]).T
# print(training_features)

# scale
training_features = preprocessing.scale(training_features)

# convert labels into integers then into column array
labels = [int(element[2]) for element in training_set]
labels = list(labels)
labels_array = np.array(labels)

ValueError: setting an array element with a sequence.

In [33]:
print "evaluating"


#evaluation
kf = KFold(len(training_set), n_folds=10)
sumf1=0
for train_index, test_index in kf:
    X_train, X_test = training_features[train_index], training_features[test_index]
    y_train, y_test = labels_array[train_index], labels_array[test_index]
    # initialize basic SVM
    classifier = svm.LinearSVC()
    # train
    classifier.fit(X_train, y_train)
    pred=classifier.predict(X_test)
    sumf1+=f1_score(pred,y_test)

print "\n\n"
print sumf1/10.0

evaluating


IndexError: index 3078 is out of bounds for axis 1 with size 7

In [2]:
import lightgbm as lgb
import random
import numpy as np
import igraph
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.cross_validation import KFold
from sklearn.metrics import f1_score
from sklearn import preprocessing
import nltk
import csv

In [10]:
data = np.random.rand(500,10) # 500 entities, each contains 10 features
label = np.random.randint(2, size=500) # binary target
test = np.random.rand(500,10)

In [11]:
train_data = lgb.Dataset(data, label=label)
test_data = lgb.Dataset(test)

In [12]:
param = {'num_leaves':31, 'num_trees':100, 'objective':'binary' }
param['metric'] = ['auc', 'binary_logloss']


In [13]:
num_round = 10
bst = lgb.train(param, train_data, num_round, valid_sets=[test_data] )

[1]	valid_0's binary_logloss: 0.694664	valid_0's auc: 1
[2]	valid_0's binary_logloss: 0.696461	valid_0's auc: 1
[3]	valid_0's binary_logloss: 0.698379	valid_0's auc: 1
[4]	valid_0's binary_logloss: 0.700209	valid_0's auc: 1
[5]	valid_0's binary_logloss: 0.702562	valid_0's auc: 1
[6]	valid_0's binary_logloss: 0.702822	valid_0's auc: 1
[7]	valid_0's binary_logloss: 0.70282	valid_0's auc: 1
[8]	valid_0's binary_logloss: 0.70517	valid_0's auc: 1
[9]	valid_0's binary_logloss: 0.705607	valid_0's auc: 1
[10]	valid_0's binary_logloss: 0.706138	valid_0's auc: 1


In [14]:
num_round = 10
lgb.cv(param, train_data, num_round, nfold=5)

{u'auc-mean': [0.57381424814771687,
  0.56404440663486233,
  0.55231772652075029,
  0.55558238395123094,
  0.54135568208998386,
  0.53715034480189816,
  0.53118672086478269,
  0.5361089873581637,
  0.53467134160866059,
  0.53401693535751971],
 u'auc-stdv': [0.073447554745996346,
  0.094320651190407875,
  0.079899987048285478,
  0.083850248636197444,
  0.073867022343390515,
  0.078378598775631941,
  0.078029607384538263,
  0.078387042964213047,
  0.079628696506225852,
  0.079220021915164807],
 u'binary_logloss-mean': [0.69141476626958298,
  0.69022148126523375,
  0.69024822127524643,
  0.6901912238164053,
  0.69185055557749764,
  0.69266288043403246,
  0.69409677245004775,
  0.69463464211811288,
  0.69507380205673652,
  0.69570159381689956],
 u'binary_logloss-stdv': [0.0029025964405674444,
  0.0053953628632738889,
  0.0069898383238011464,
  0.0092156649059864735,
  0.0099668742901471083,
  0.011332094967849322,
  0.013156231250234334,
  0.014731229422817805,
  0.015153407308638409,
  0.

In [17]:
ypred = bst.predict(test)

In [16]:
ypred

array([ 0.50299316,  0.53623774,  0.53623957,  0.46762597,  0.53366352,
        0.41448527,  0.53794684,  0.57103254,  0.49149841,  0.47795521,
        0.54680136,  0.40975793,  0.48521382,  0.48041589,  0.54509733,
        0.53623957,  0.49149841,  0.49016921,  0.47795521,  0.49149841,
        0.55835475,  0.38283595,  0.49149841,  0.46199891,  0.55862717,
        0.53623774,  0.48521727,  0.40975793,  0.37299788,  0.40008384,
        0.5487641 ,  0.41245765,  0.5487641 ,  0.49149841,  0.5487641 ,
        0.55046572,  0.50959954,  0.47942992,  0.53623774,  0.54509733,
        0.50310555,  0.48521382,  0.38283595,  0.57103254,  0.54680136,
        0.49543731,  0.49720201,  0.51592545,  0.47795521,  0.53991748,
        0.46509217,  0.50959954,  0.40008384,  0.5487641 ,  0.40008384,
        0.53366352,  0.50959954,  0.57103254,  0.44444312,  0.5487641 ,
        0.503109  ,  0.57135651,  0.49149841,  0.50928021,  0.50959954,
        0.49016921,  0.47795521,  0.53991748,  0.49016921,  0.51

In [18]:
test

array([[ 0.8584349 ,  0.37642426,  0.6933843 , ...,  0.46964195,
         0.59748801,  0.17279788],
       [ 0.11108495,  0.0735061 ,  0.16041545, ...,  0.55415686,
         0.74428731,  0.34377489],
       [ 0.5375594 ,  0.81558312,  0.53612251, ...,  0.44787832,
         0.96061597,  0.68550323],
       ..., 
       [ 0.36815489,  0.49801124,  0.06919135, ...,  0.92552546,
         0.2068397 ,  0.99926708],
       [ 0.22164502,  0.96720609,  0.88286236, ...,  0.10121715,
         0.72455951,  0.7782057 ],
       [ 0.39687975,  0.06862831,  0.06990003, ...,  0.46480213,
         0.2148502 ,  0.55232971]])